# Hyperparameters optimization with Optuna
Date : 10/08/2020

Study the hyperparameters and their best values

In [1]:
import optuna
import neptune
NEPTUNE_API_TOKEN="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiMGRkNDk3ZWEtZjYyOC00ODYwLWFhZTQtZTY0YmI5YzA5NWZjIn0="
import neptunecontrib.monitoring.optuna as opt_utils
import plotly
from voicemap.train import *

In [2]:
# useful imports
from IPython.display import SVG, Audio, display
import subprocess
import pandas as pd
import numpy as np
from tqdm import tqdm
import librosa
%matplotlib inline
import os
from matplotlib import pyplot as plt

# voicemap imports
from config import PATH, DATA_PATH, LOG_PATH

In [3]:
t = training_args()
t.lr = 0.01
t.filters = 64
t.epochs = 7
t.train_datasets_letters = "LCT"
t.val_datasets_letters = "LSCT"
print(vars(t))

{'model': 'resnet', 'model_path': None, 'dim': 1, 'lr': 0.01, 'weight_decay': 0.01, 'momentum': 0.9, 'epochs': 7, 'filters': 64, 'batch_size': 1500, 'n_seconds': 3, 'downsampling': 4, 'use_standardized': True, 'spectrogram': True, 'precompute_spect': True, 'window_length': 0.02, 'window_hop': 0.01, 'device': 'cuda', 'n_t': 0, 'T': None, 'n_f': 0, 'F': None, 'train_datasets_letters': 'LCT', 'val_datasets_letters': 'LSCT'}


In [4]:
#train(args=t, t.train_datasets_letters, t.val_datasets_letters)

In [5]:
def objective(trial: optuna.trial.Trial):
    t.lr = trial.suggest_categorical("lr", [0.1, 0.01])
    t.filters = trial.suggest_categorical("filters", [64, 128])
    t.batch_size = trial.suggest_categorical("batch_size", [128, 32, 1])
    t.epochs = trial.suggest_int("epochs", 1, 5)
    t.weight_decay = trial.suggest_loguniform("weight_decay", 0.005, 0.95)
    t.momentum = trial.suggest_uniform("momentum", 0, 1)
    #t.spectrogram = trial.suggest_int("spectrogram", 0, 1)
    #t.window_length = trial.suggest_uniform("window_length", 0.01, 0.5)
    #t.window_hop = trial.suggest_uniform("window_hop", 0.01, 0.5)
    # t.n_seconds = trial.suggest_uniform("n_seconds", 2.5, 3) # must refactor spectrograms generation before pushing n_seconds above 3
    t.train_datasets_letters = trial.suggest_categorical("train_datasets_letters", ["LCT", "CT"])
    print(vars(t))
    epochs_results = train(args=t, train_datasets_letters=t.train_datasets_letters, val_datasets_letters=t.val_datasets_letters, optunaTrial=trial, verbose=True)
    print(epochs_results)
    result = epochs_results[-1]['val_loss']
    return result  

In [ ]:
# Setup neptune
neptune.init('team-pep/speaker-identification', api_token=NEPTUNE_API_TOKEN)
neptune_callback = opt_utils.NeptuneCallback(log_charts=True, log_study=True)
neptune.create_experiment(params=t.__dict__)
# Setup optuna
study = optuna.load_study(study_name='study-01', storage=f"sqlite:///{LOG_PATH}/optimization.db")

# Optuna optimization
study.optimize(objective, n_trials=5, callbacks=[neptune_callback])
# Log into neptune
#neptune.log_metric("val_loss", study.best_value)
opt_utils.log_study_info(study)

https://ui.neptune.ai/team-pep/speaker-identification/e/SID-9
{'model': 'resnet', 'model_path': None, 'dim': 1, 'lr': 0.1, 'weight_decay': 0.001373775178985992, 'momentum': 0.9240961567020923, 'epochs': 10, 'filters': 128, 'batch_size': 1, 'n_seconds': 3, 'downsampling': 4, 'use_standardized': True, 'spectrogram': True, 'precompute_spect': True, 'window_length': 0.02, 'window_hop': 0.01, 'device': 'cuda', 'n_t': 0, 'T': None, 'n_f': 0, 'F': None, 'train_datasets_letters': 'CT', 'val_datasets_letters': 'LSCT'}
Total no. speakers = 1031
Begin training...


100%|██████████| 250/250 [01:49<00:00,  2.31it/s]

 31%|███       | 77/250 [00:35<01:15,  2.29it/s]


100%|██████████| 250/250 [01:51<00:00,  2.31it/s]

 62%|██████▏   | 156/250 [01:10<00:41,  2.29it/s]


100%|██████████| 250/250 [01:51<00:00,  2.31it/s]


ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.


...

100%|██████████| 250/250 [01:51<00:00,  2.32it/s]


ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.



100%|██████████| 250/250 [01:50<00:00,  2.31it/s]



Epoch 00007: val_loss did not improve from 731.43126


Epoch 7: 100%|█████████▉| 27836/27837 [1:35:26<00:00,  4.87it/s, loss=18.8, accuracy=0]

In [ ]:
study = optuna.load_study(study_name='study-01', storage=f"sqlite:///{LOG_PATH}/optimization.db")
study.get_trials()

In [3]:


# log some metrics

for i in range(100):
    neptune.log_metric('loss', 0.95**i)

neptune.log_metric('AUC', 0.96)

https://ui.neptune.ai/iribarnesy/sandbox/e/SAN-1
